In [1]:
import math
import pandas as pd
import numpy as np
from scipy import optimize 
from sklearn.neural_network import MLPRegressor

In [2]:
data = pd.read_csv('workouts.csv')
data.describe()
Actual_Performance = data['IF']
Offset_Performance = []
for i in range(len(Actual_Performance)):
    Offset_Performance.append(np.mean(Actual_Performance[i: (i+28)])) 
TSS = data['TSS']
Block_TSS = [] 
for i in range(len(TSS)):
    avg_TSS = np.mean(TSS[i:(i+28)])
    Block_TSS.append(avg_TSS)


In [3]:
def calc_vo2_if_bike(row, max_hr, resting_hr, weight) :
    if row['WorkoutType'] == 'Bike':
        percent_vo2 = (row['HeartRateAverage'] - resting_hr) / (max_hr - resting_hr)
        vo2_power = row['PowerAverage'] / percent_vo2
        vo2_estimated = (((vo2_power)/75)*1000)/weight
        return vo2_estimated


In [23]:
def optimize_banister(params):
    data = pd.read_csv('workouts.csv')
    data['day_TSS'] = data['TSS'].groupby(data['WorkoutDay']).transform('sum') #Fill in any missing days with zero
    data['day_TSS'] = data['day_TSS'].fillna(0) #Fill in any missing days with zero
    data['bike_V02'] = data.apply(lambda row: calc_vo2_if_bike(row, 196, 50, 74), axis=1) #Calculate VO2 for each bike workout
    data = data[['WorkoutDay', 'day_TSS', 'bike_V02']] #Keep only the columns we need
    data = data.groupby('WorkoutDay').mean() #Group by day and take the mean of the TSS and VO2
    data['WorkoutDate'] = data.index #Add a column for the date
    data['WorkoutDate'] = pd.to_datetime(data['WorkoutDate']) #Convert the date column to a datetime object
    data = data.sort_values(by=['WorkoutDate']) #Sort the data by date
    data.index = pd.DatetimeIndex(data['WorkoutDate']) #Set the index to the date
    missing_dates = pd.date_range(start=data.index.min(), end=data.index.max()) #Create a list of all the dates in the range
    data = data.reindex(missing_dates, fill_value=0) #Add missing dates
    data['bike_V02'] = data['bike_V02'].fillna(method="ffill") #FiLL missing VOZ values with previous value
    data = data.dropna() #Drop any remaining rows with missing values TSS= data['day_TSS'].to_list() #Convert the TSS column to a list
    TSS = data['day_TSS'].to_list() #Convert the TSS column to a list
    Performance = data['bike_V02'].to_list() #Convert the V02 column to a list
    losses = [] #Create an empty list to store the loss values from our model
    ctls = [0] #Create an empty list to store the CTL values from our model with starting CTL of 0
    atls = [0] #Create an empty list to store the ATL values from our model with starting ATL of 0 for i in range(len(TSS)):
    for i in range(len(TSS)):
        ctl = (TSS[i] * (1-math.exp(-1/params[3]))) + (ctls[i] * (math.exp(-1/params[3]))) #Calculate the CTL for the day
        atl = (TSS[i] * (1-math.exp(-1/params[4]))) + (atls[i] * (math.exp(-1/params[4]))) #Calculate the ATL for the day
        ctls.append(ctl) #Add the CTL to the list
        atls.append(atl) #Add the ATL to the List
        Banister_Prediction = params[2] + params[0]*ctl - params[1]*atl #Calculate the Banister Prediction for the day
        loss = abs(Performance[i]- Banister_Prediction)
        losses.append(loss) #Add the loss to the list
    # print(f"CTLs: {ctls}")
    # print(f"ATLS: {atls}")
    MAE= np.mean(losses) #Calculate the mean absolute error
    # print(f"MAE: {MAE}")
    return MAE

In [24]:
initial_guess = [0.1, 0.5, 50, 40, 15]
individual_banister_model = optimize.minimize(optimize_banister,  x0 = initial_guess, bounds=[(0,1),(0,1),(20,50),(20,60),(10,20)])
print(individual_banister_model)

# Individual Neural Network Model
individual_neural_net_model = MLPRegressor(solver='lbfgs', activation='relu', hidden_layer_sizes=[50], random_state=42)
print(Block_TSS)
print(Offset_Performance)

      fun: 4.039287638445985
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.80849452, -0.23379281,  0.08571436,  0.00119105,  0.00272697])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 426
      nit: 13
     njev: 71
   status: 0
  success: True
        x: array([ 0.09719483,  0.15562943, 49.99895401, 39.98851172, 15.04958741])
[57.281739130434794, 57.57434782608696, 57.28217391304348, 56.01739130434782, 55.74521739130433, 58.35708333333332, 56.831666666666656, 57.107916666666654, 57.270833333333336, 59.6856, 59.00807692307693, 59.00807692307693, 59.040740740740745, 58.70407407407407, 62.684814814814814, 61.27346153846155, 62.284400000000005, 61.50240000000001, 61.93120000000001, 63.918, 65.06208333333335, 62.82708333333334, 65.46708333333333, 62.41041666666666, 62.73208333333332, 63.838695652173904, 60.6691304347826, 60.62608695652173, 60.85956521739129, 59.76782608695652, 64.01608695652173, 62.37260869565217, 63.31086956521739, 

In [34]:
arr = np.array(Block_TSS)
arr_Block_TSS = arr.reshape(-1,1)
# print(arr_Block_TSS)

individual_neural_net_model.fit(arr_Block_TSS, Offset_Performance)
# print(individual_neural_net_model.predict(arr_Block_TSS))
# individual_neural_net_model_score = individual_neural_net_model.score(arr_Block_TSS, Offset_Performance)
# individual_neural_net_model.fit([[]100],[0.5])




MLPRegressor(hidden_layer_sizes=[50], random_state=42, solver='lbfgs')